# MVA - Homework 1 - Reinforcement Learning (2022/2023)

**Name:** DE SURREL Thibault

## Instructions

* The deadline is **November 10 at 11:59 pm (Paris time).**

* By doing this homework you agree to the late day policy, collaboration and misconduct rules reported on [Piazza](https://piazza.com/class/l4y5ubadwj64mb/post/6).

* **Mysterious or unsupported answers will not receive full credit**. A correct answer, unsupported by calculations, explanation, or algebraic work will receive no credit; an incorrect answer supported by substantially correct calculations and explanations might still receive partial credit.

* Answers should be provided in **English**.

# Colab setup

In [ ]:
from IPython import get_ipython

if 'google.colab' in str(get_ipython()):
  # install rlberry library
  !pip install git+https://github.com/rlberry-py/rlberry.git@mva2021#egg=rlberry[default] > /dev/null 2>&1

  # install ffmpeg-python for saving videos
  !pip install ffmpeg-python > /dev/null 2>&1

  # packages required to show video
  !pip install pyvirtualdisplay > /dev/null 2>&1
  !apt-get install -y xvfb python-opengl ffmpeg > /dev/null 2>&1

  print("Libraries installed, please restart the runtime!")


In [ ]:
# Create directory for saving videos
!mkdir videos > /dev/null 2>&1

# Initialize display and import function to show videos
import rlberry.colab_utils.display_setup
from rlberry.colab_utils.display_setup import show_video

In [ ]:
# Useful libraries
import numpy as np
import matplotlib.pyplot as plt

# Preparation

In the coding exercises, you will use a *grid-world* MDP, which is represented in Python using the interface provided by the [Gym](https://gym.openai.com/) library. The cells below show how to interact with this MDP and how to visualize it.


In [ ]:
from rlberry.envs import GridWorld

def get_env():
  """Creates an instance of a grid-world MDP."""
  env = GridWorld(
      nrows=5,
      ncols=7,
      reward_at = {(0, 6):1.0},
      walls=((0, 4), (1, 4), (2, 4), (3, 4)),
      success_probability=0.9,
      terminal_states=((0, 6),)
  )
  return env

def render_policy(env, policy=None, horizon=50):
  """Visualize a policy in an environment

  Args:
    env: GridWorld
        environment where to run the policy
    policy: np.array
        matrix mapping states to action (Ns).
        If None, runs random policy.
    horizon: int
        maximum number of timesteps in the environment.
  """
  env.enable_rendering()
  state = env.reset()                       # get initial state
  for timestep in range(horizon):
      if policy is None:
        action = env.action_space.sample()  # take random actions
      else:
        action = policy[state]
      next_state, reward, is_terminal, info = env.step(action)
      state = next_state
      if is_terminal:
        break
  # save video and clear buffer
  env.save_video('./videos/gw.mp4', framerate=5)
  env.clear_render_buffer()
  env.disable_rendering()
  # show video
  show_video('./videos/gw.mp4')


In [ ]:
# Create an environment and visualize it
env = get_env()
render_policy(env)  # visualize random policy

# The reward function and transition probabilities can be accessed through
# the R and P attributes:
print(f"Shape of the reward array = (S, A) = {env.R.shape}")
print(f"Shape of the transition array = (S, A, S) = {env.P.shape}")
print(f"Reward at (s, a) = (1, 0): {env.R[1, 0]}")
print(f"Prob[s\'=2 | s=1, a=0]: {env.P[1, 0, 2]}")
print(f"Number of states and actions: {env.Ns}, {env.Na}")

# The states in the griworld correspond to (row, col) coordinates.
# The environment provides a mapping between (row, col) and the index of
# each state:
print(f"Index of state (1, 0): {env.coord2index[(1, 0)]}")
print(f"Coordinates of state 5: {env.index2coord[5]}")

# Part 1 - Dynamic Programming

## Question 1.1

Consider a general MDP with a discount factor of $\gamma < 1$. Assume that the horizon is infinite (so there is no termination). A policy $\pi$ in this MDP
induces a value function $V^\pi$. Suppose an affine transformation is applied to the reward, what is
the new value function? Is the optimal policy preserved?



### **Answer**

Let $f \colon x \mapsto ax + b$ be the affine transformation with $a,b \in \mathbb{R}$. 
Then, the new value fonction $V^\pi_f$ is : $$V^\pi_f(s) = \mathbb{E}\left[\sum\limits_{t = 0}^\infty \gamma^t f(r(s_t,a_t)) \mid s_0 = s, a_t \sim d_t(h_t), \pi \right] = a V^\pi(s) + \frac{b}{1-\gamma}$$
If $a > 0$, the optimal policy is preserved because the $argmax$ is the same. 
If $a \leq 0$, the optimal policy is not preserved.

## Question 1.2

Consider an infinite-horizon $\gamma$-discounted MDP. We denote by $Q^*$ the $Q$-function of the optimal policy $\pi^*$. Prove that, for any function $Q(s, a)$ (which is **not** necessarily the value function of a policy), the following inequality holds for any state $s$:

$$
V^{\pi_Q}(s) \geq V^*(s) - \frac{2}{1-\gamma}||Q^*-Q||_\infty,
$$

where $||Q^*-Q||_\infty = \max_{s, a} |Q^*(s, a) - Q(s, a)|$ and $\pi_Q(s) \in \arg\max_a Q(s, a)$. Can you use this result to show that any policy $\pi$ such that $\pi(s) \in \arg\max_a Q^*(s, a)$ is optimal?

### **Answer**

Let $s$ be a state. We have :
\begin{equation}
\begin{aligned}
Q^*(s,\pi^*(s)) - Q^*(s,\pi_Q(s)) &= Q^*(s,\pi^*(s)) - Q(s,\pi^*(s)) + Q(s,\pi^*(s)) - Q^*(s,\pi_Q(s)) \\
&\leq \|Q^* - Q \|_{\infty} + Q(s,\pi^*(s)) - Q^*(s,\pi_Q(s))
\end{aligned}
\end{equation}
By definition of $\pi_Q(s)$, we have that 
$Q(s,\pi^*(s)) \leq Q(s,\pi_Q(s))$ so, 
\begin{equation}
\begin{aligned}
Q^*(s,\pi^*(s)) - Q^*(s,\pi_Q(s)) &\leq \|Q^* - Q \|_{\infty} + Q(s,\pi_Q(s)) - Q^*(s,\pi_Q(s)) \\
&\leq  2 \|Q^* - Q \|_{\infty}
\end{aligned}
\end{equation}

Let $Q^{\pi_Q}$ be the $Q$-function of the policy $\pi_Q$. We then remark that we have, thank to the Bellman equation : 
\begin{equation}
\begin{aligned}
Q^*(s,\pi^*(s)) &= r(s,\pi^*(s)) + \gamma \sum_{s'}p(s,\pi^*(s),s')V^*(s') = V^*(s) \\
Q^{\pi_Q}(s,\pi_Q(s)) &= r(s,\pi_Q(s)) + \gamma \sum_{s'}p(s,\pi_Q(s),s')V^{\pi_Q}(s') = V^{\pi_Q}(s)
\end{aligned}
\end{equation}

So, when considering the first inequality shown :  
\begin{equation}
\begin{aligned}
2 \|Q^* - Q \|_{\infty} &\geq Q^*(s,\pi^*(s)) - Q^*(s,\pi_Q(s)) \\
&\geq Q^*(s,\pi^*(s)) - Q^{\pi_Q}(s,\pi_Q(s)) + Q^{\pi_Q}(s,\pi_Q(s)) - Q^*(s,\pi_Q(s)) \\
&\geq V^*(s) - V^{\pi_Q}(s) + r(s,\pi_Q(s)) + \gamma \sum_{s'}p(s,\pi_Q(s),s')V^{\pi_Q}(s') - r(s,\pi_Q(s)) + \gamma \sum_{s'}p(s,\pi_Q(s),s')V^*(s') \\
&\geq  V^*(s) - V^{\pi_Q}(s) + \gamma\sum_{s'}p(s,\pi_Q(s),s')[V^{\pi_Q}(s') - V^*(s')]
\end{aligned}
\end{equation}

Let $s_0$ be a state such that $V^*(s_0) - V^{\pi_Q}(s_0) = \max_s V^*(s) - V^{\pi_Q}(s)$. Then, for all states $s$, we have $V^*(s_0) - V^{\pi_Q}(s_0) \leq V^{\pi_Q}(s) - V^*(s)$. Especially, ther former inequality holds for all states $s$, we have :  
\begin{equation}
\begin{aligned}
2 \|Q^* - Q \|_{\infty} &\geq  V^*(s_0) - V^{\pi_Q}(s_0) + \gamma \sum_{s'}p(s_0,\pi_Q(s_0),s')[V^{\pi_Q}(s') - V^*(s')]\\
&\geq V^*(s_0) - V^{\pi_Q}(s_0) + \gamma \left[\sum_{s'}p(s_0,\pi_Q(s_0),s')\right](V^{\pi_Q}(s_0) - V^*(s_0)) \\
&\geq (1 - \gamma )(V^*(s_0) - V^{\pi_Q}(s_0))
\end{aligned}
\end{equation}
We then have the result, for all state $s$ : 
$$ V^*(s) - V^{\pi_Q}(s) \leq V^*(s_0) - V^{\pi_Q}(s_0) \leq \frac{2}{1-\gamma}\|Q^* - Q \|_{\infty} $$
Finally : 
$$V^{\pi_Q}(s) \geq V^*(s) - \frac{2}{1-\gamma}\|Q^* - Q \|_{\infty}$$


 Let $\pi$ be a policy such that $\pi(s) \in \arg\max_a Q^*(s, a)$. Then, with previous notations, we have $\pi = \pi_{Q^*}$. So, by the former inequality : 
$$V^{\pi}(s) = V^{\pi_{Q^*}}(s) \geq V^*(s) - \frac{2}{1-\gamma}\|Q^* - Q^* \|_{\infty} \geq V^*(s)$$ 
The policy is therefor optimal.  

## Question 1.3

In this question, you will implement and compare the policy and value iteration algorithms for a finite MDP. 

Complete the functions `policy_evaluation`, `policy_iteration` and `value_iteration` below.


Compare value iteration and policy iteration. Highlight pros and cons of each method.

### **Answer**


In [ ]:
def policy_evaluation(P, R, policy, gamma=0.9, tol=1e-2):
    """
    Args:
        P: np.array
            transition matrix (NsxNaxNs)
        R: np.array
            reward matrix (NsxNa)
        policy: np.array
            matrix mapping states to action (Ns)
        gamma: float
            discount factor
        tol: float
            precision of the solution
    Return:
        value_function: np.array
            The value function of the given policy
    """
    n = 1000
    Ns, Na = R.shape
    value_function = np.zeros(Ns)
    # ====================================================

    delta = 1 + tol
    while delta > tol:
      delta = 0
      for s in range(Ns):
        a = policy[s]

        new_value = R[s,a] + gamma * np.sum([P[s,a,s_p]*value_function[s_p] for s_p in range(Ns)])
        delta = max(delta,np.abs(new_value - value_function[s]))
        value_function[s] = new_value

    # ====================================================
    return value_function

In [ ]:
def policy_iteration(P, R, gamma=0.9, tol=1e-3):
    """
    Args:
        P: np.array
            transition matrix (NsxNaxNs)
        R: np.array
            reward matrix (NsxNa)
        gamma: float
            discount factor
        tol: float
            precision of the solution
    Return:
        policy: np.array
            the final policy
        V: np.array
            the value function associated to the final policy
        number_it : int
            The number of iterations it took to compute the value fonction
    """
    Ns, Na = R.shape
    old_V = np.zeros(Ns)
    policy = np.ones(Ns, dtype=int)
    number_it = 0
    # ====================================================
	  # YOUR IMPLEMENTATION HERE 
    #
    V = policy_evaluation(P, R, policy, gamma=gamma, tol=tol)
    
    delta = tol + 1

    while delta > tol:
      delta = 0
      number_it += 1
      old_V = np.copy(V)
      policy = np.array([np.argmax(R[s] + gamma*np.dot(P[s],V)) for s in range(Ns)])

      V = policy_evaluation(P, R, policy, gamma=gamma, tol=tol)
      
      delta = np.linalg.norm(V - old_V)
    # ====================================================
    return policy, V, number_it

In [ ]:
def value_iteration(P, R, gamma=0.9, tol=1e-3):
    """
    Args:
        P: np.array
            transition matrix (NsxNaxNs)
        R: np.array
            reward matrix (NsxNa)
        gamma: float
            discount factor
        tol: float
            precision of the solution
    Return:
        Q: final Q-function (at iteration n)
        greedy_policy: greedy policy wrt Qn
        Qfs: all Q-functions generated by the algorithm (for visualization)
        number_it : int
            The number of iterations it took to compute the value fonction
    """
    Ns, Na = R.shape
    Q = np.zeros((Ns, Na))
    Qfs = [Q]
    number_it = 0
    # ====================================================
    delta = 1 + tol
    while delta > tol:
      number_it += 1
      Q_new = R + gamma * np.dot(P,np.max(Q,axis=1)) 
      delta = np.amax(np.abs(Q-Q_new))
      Q = np.copy(Q_new)
      Qfs.append(np.copy(Q_new))
    
    greedy_policy = np.argmax(Q, axis=1)
    # ====================================================
    return Q, greedy_policy, Qfs, number_it

### Testing your code

In [ ]:
from time import time 
# Parameters
tol = 1e-5
gamma = 0.99

# Environment
env = get_env()

# run value iteration to obtain Q-values
t = time()
VI_Q, VI_greedypol, all_qfunctions, number_it_VI = value_iteration(env.P, env.R, gamma=gamma, tol=tol)
print("Time to compute the value iteration : " + str(time() - t ))
print("Number of iterations for the value iteration : " + str(number_it_VI))

# render the policy
print("[VI]Greedy policy: ")
render_policy(env, VI_greedypol)

# compute the value function of the greedy policy using matrix inversion
# ====================================================
greedy_V = policy_evaluation(env.P, env.R, VI_greedypol, gamma= gamma, tol= tol )

# ====================================================

# show the error between the computed V-functions and the final V-function
# (that should be the optimal one, if correctly implemented)
# as a function of time
final_V = all_qfunctions[-1].max(axis=1)
norms = [ np.linalg.norm(q.max(axis=1) - final_V) for q in all_qfunctions]
plt.plot(norms)
plt.xlabel('Iteration')
plt.ylabel('Error')
plt.title("Value iteration: convergence")

#### POLICY ITERATION ####
t = time()

PI_policy, PI_V, number_it_PI = policy_iteration(env.P, env.R, gamma=gamma, tol=tol)
print("Time to compute the poilcy iteration : " + str(time() - t ))
print("Number of iterations for the policy iteration : " + str(number_it_PI))

print("\n[PI]final policy: ")
render_policy(env, PI_policy)

## Uncomment below to check that everything is correct
assert np.allclose(PI_policy, VI_greedypol),\
     "You should check the code, the greedy policy computed by VI is not equal to the solution of PI"
assert np.allclose(PI_V, greedy_V),\
     "Since the policies are equal, even the value function should be"

plt.show()

**Answer**

**In theory** we have the following pros et cons for each method : 

|                    **Policy iteration**                   |                    **Value iteration**                   |
|:---------------------------------------------------------:|:--------------------------------------------------------:|
| Algorithm is more complex since we need policy evaluation | Simpler algorithm                                        |
| Cheaper to compute                                        | More expensive to compute since we have a max to compute |
| Faster to converge                                        | Slower to converge                                       |
| The convergence is guaranteed                             | The convergence is guaranteed                            |

**In practice** we can see that the value iteration is a lot quicker than the policy iteration ($\sim$ 0.04 secondes for the value iteration versus 4.8 secondes for the policy iteration). But we also see that the policy iteration needs a lot less iterations (only 4) to converge versus the value iteration (1147 iterations). In this example, the computation time of each iteration is thus a lot quicker for value iteration than for policy iteraiton. In fact, at each step, the policy iteration needs to compute a policy evaluation, and in this take, computing an $argmax$ like in the value iteration is quicker. 

# Part 2 - Tabular RL

## Question 2.1

The code below collects two datasets of transitions (containing states, actions, rewards and next states) for a discrete MDP.

For each of the datasets:

1. Estimate the transitions and rewards, $\hat{P}$ and $\hat{R}$.
2. Compute the optimal value function and the optimal policy with respect to the estimated MDP (defined by $\hat{P}$ and $\hat{R}$), which we denote by $\hat{\pi}$ and $\hat{V}$.
3. Numerically compare the performance of $\hat{\pi}$ and $\pi^\star$ (the true optimal policy), and the error between $\hat{V}$ and $V^*$ (the true optimal value function).

Which of the two data collection methods do you think is better? Why?

### **Answer**

[answer last question + implementation below]

In [ ]:
def get_random_policy_dataset(env, n_samples):
  """Get a dataset following a random policy to collect data."""
  states = []
  actions = []
  rewards = []
  next_states = []
  
  state = env.reset()
  for _ in range(n_samples):
    action = env.action_space.sample()
    next_state, reward, is_terminal, info = env.step(action)
    states.append(state)
    actions.append(action)
    rewards.append(reward)
    next_states.append(next_state)
    # update state
    state = next_state
    if is_terminal:
      state = env.reset()

  dataset = (states, actions, rewards, next_states)
  return dataset

def get_uniform_dataset(env, n_samples):
  """Get a dataset by uniformly sampling states and actions."""
  states = []
  actions = []
  rewards = []
  next_states = []
  for _ in range(n_samples):
    state = env.observation_space.sample()
    action = env.action_space.sample()
    next_state, reward, is_terminal, info = env.sample(state, action)
    states.append(state)
    actions.append(action)
    rewards.append(reward)
    next_states.append(next_state)

  dataset = (states, actions, rewards, next_states)
  return dataset


# Collect two different datasets
num_samples = 500
env = get_env()
dataset_1 = get_random_policy_dataset(env, num_samples)
dataset_2 = get_uniform_dataset(env, num_samples)


# Item 3: Estimate the MDP with the two datasets; compare the optimal value
# functions in the true and in the estimated MDPs

# ...

In [ ]:
Ns, Na = env.R.shape

In [ ]:
def estimate_p(dataset, Ns, Na, num_samples):
  (states, actions, rewards, next_states) = dataset
  N = np.zeros((Ns,Na,Ns))  
  p_hat = np.zeros((Ns,Na,Ns))
  for i in range(num_samples):
    N[states[i],actions[i],next_states[i]] += 1
  
  for s in range(Ns):
    for a in range(Na):
      div = np.sum([N[s,a,i] for i in range(Ns)])
      if div != 0:
          p_hat[s,a] =  N[s,a]/div
  return p_hat

def estimate_r(dataset, Ns, Na, num_samples):
  (states, actions, rewards, next_states) = dataset
  r_hat = np.zeros((Ns,Na))
  N = np.zeros((Ns,Na,Ns))  
  for i in range(num_samples):
    N[states[i],actions[i],next_states[i]] += 1
    r_hat[states[i],actions[i]] += rewards[i]
  
  for s in range(Ns):
    for a in range(Na):
      div = np.sum([N[s,a,i] for i in range(Ns)])
      if div != 0:
          r_hat[s,a] /= div
  return r_hat


In [ ]:
from sklearn.metrics import mean_squared_error, mean_absolute_error
#For dataset_1
p_hat_1 = estimate_p(dataset_1, Ns, Na, num_samples)
r_hat_1 = estimate_r(dataset_1, Ns, Na, num_samples)

pi_hat_1, V_hat_1, _ = policy_iteration(p_hat_1, r_hat_1)
pi_star_1, V_star_1, _ = policy_iteration(env.P,env.R)

mae_1_V = mean_absolute_error(V_hat_1,V_star_1)  
mse_1_V = mean_squared_error(V_hat_1,V_star_1)  

#For dataset_2
p_hat_2 = estimate_p(dataset_2, Ns, Na, num_samples)
r_hat_2 = estimate_r(dataset_2, Ns, Na, num_samples)

pi_hat_2, V_hat_2, _ = policy_iteration(p_hat_2, r_hat_2)
pi_star_2, V_star_2, _ = policy_iteration(env.P,env.R)

mae_2_V = mean_absolute_error(V_hat_2,V_star_2)  
mse_2_V = mean_squared_error(V_hat_2,V_star_2)  

print("Results for dataset_1 : ")
print("Mean square error for V : ", mse_1_V)
print("Mean absolute error for V : ", mae_1_V)


print("Results for dataset_2 : ")
print("Mean square error for V : ", mse_2_V)
print("Mean absolute error for V : ", mae_2_V)


**Answer**

Intuitively, the second dataset is better than the first one. In fact, the uniform sampling chooses a random state and action at each step whereas the random policy datasat chooses a random policy and follows it. So the uniform sampling allows us to explore more datapoints. 

The results of the experiments confirm this idea. In fact, the mean absolute error and the mean squared error are a lot better for dataset 2 than dataset 1.

## Question 2.2

Suppose that $\hat{P}$ and $\hat{R}$ are estimated from a dataset of exactly $N$ i.i.d. samples from **each** state-action pair. This means that, for each $(s,a)$, we have $N$ samples $\{(s_1',r_1, \dots, s_N', r_N\}$, where $s_i' \sim P(\cdot | s,a)$ and $r_i \sim R(s,a)$ for $i=1,\dots,N$, and
$$ \hat{P}(s'|s,a) = \frac{1}{N}\sum_{i=1}^N \mathbb{1}(s_i' = s'), $$
$$ \hat{R}(s,a) = \frac{1}{N}\sum_{i=1}^N r_i.$$
Suppose that $R$ is a distribution with support in $[0,1]$. Let $\hat{V}$ be the optimal value function computed in the empirical MDP (i.e., the one with transitions $\hat{P}$ and rewards $\hat{R}$). For any $\delta\in(0,1)$, derive an upper bound to the error

$$ \| \hat{V} - V^* \|_\infty $$

which holds with probability at least $1-\delta$.

**Note** Your bound should only depend on deterministic quantities like $N$, $\gamma$, $\delta$, $S$, $A$. It should *not* dependent on the actual random samples.

**Hint** The following two inequalities may be helpful.

1. **A (simplified) lemma**. For any state $\bar{s}$,

$$ |\hat{V}(\bar{s}) - V^*(\bar{s})| \leq \frac{1}{1-\gamma}\max_{s,a} \left| R(s,a) - \hat{R}(s,a) + \gamma \sum_{s'}(P(s'|s,a) - \hat{P}(s'|s,a)) V^*(s') \right|$$

2. **Hoeffding's inequality**. Let $X_1, \dots X_N$ be $N$ i.i.d. random variables bounded in the interval $[0,b]$ for some $b>0$. Let $\bar{X} = \frac{1}{N}\sum_{i=1}^N X_i$ be the empirical mean. Then, for any $\epsilon > 0$,

$$ \mathbb{P}(|\bar{X} - \mathbb{E}[\bar{X}]| > \epsilon) \leq 2e^{-\frac{2N
\epsilon^2}{b^2}}.$$

### **Answer**

Let $(s,a)$ be a state-action pair. We then define $X_i^{s,a}$ by : 
$$X_i^{s,a} = r_i + \gamma \sum\limits_{s'} \mathbb{1}_{s_i' = s'}V^*(s')$$
The $(X_i^{s,a})_{i}$ are $N$ i.i.d. random variables because $r_i$ and $s_i'$ are. There are bounded in the the interval $[0, \frac{1}{1-\gamma}]$. In fact, we have $|r_i| \leq 1$ so :
$$|X_i^{s,a}| \leq |r_i| +  \gamma \sum\limits_{s'} \mathbb{1}_{s_i' = s'}|V^*(s)| \leq 1 +  \gamma \sum\limits_{s'}|V^*(s')|$$
Moreover 
$$|V^*(s)| \leq \mathbb{E}\left[\sum\limits_{t=0}^{\infty} \gamma^t |R(s_t,a_t)| \mid s_0 = s\right] \leq \sum\limits_{t=0}^{\infty} \gamma^t \leq \frac{1}{1-\gamma}$$
So, we have $|X_i^{a,b}| \leq 1 + \frac{\gamma}{1-\gamma} =  \frac{1}{1-\gamma}$

Denoting $\bar{X}^{s,a} = \frac{1}{N}\sum\limits_{i=1}^N X_i^{s,a}$, we have 
$$\bar{X}^{s,a} = \frac{1}{N}\sum\limits_{i=1}^N r_i + \gamma \sum\limits_{s'}\frac{1}{N}\sum\limits_{i=1}^N \mathbb{1}_{s_i' = s'}V^*(s') = \hat{R}(s,a) + \gamma\sum\limits_{s'} \hat{P}(s'\mid s,a) V^*(s') $$
and 
$$\mathbb{E}[\bar{X}^{s,a}] = R(s,a) + \gamma\sum\limits_{s'} P(s'\mid s,a) V^*(s') $$
So, we have 
$$|\bar{X}^{s,a} - \mathbb{E}[\bar{X}^{s,a}]| = | \hat{R}(s,a) - R(s,a) + \gamma \sum\limits_{s'} (\hat{P}(s'\mid s,a) - P(s'\mid s,a) ) V^*(s')|$$
We can now use Hoeffding's inequality : for $\varepsilon > 0$ to be determined,
$$\mathbb{P}\left( |\bar{X}^{s,a} - \mathbb{E}[\bar{X}^{s,a}]| \leq \varepsilon\right) \geq 1 - 2e^{-2(1-\gamma)^2N\varepsilon^2}$$
Then, using le lemme we have, for all state $\bar{s}$, 
$$\|\hat{V} - V^* \|_{\infty} \leq |\hat{V}(\bar{s}) - V^*(\bar{s})| \leq \frac{1}{1-\gamma}\max_{s,a} |\bar{X}^{s,a} - \mathbb{E}[\bar{X}^{s,a}]| $$
So, we have :
$$\mathbb{P}\left(\|\hat{V} - V^* \|_{\infty} \leq  \varepsilon\right) \geq \mathbb{P}\left(\frac{1}{1-\gamma}\max_{s,a} |\bar{X}^{s,a} - \mathbb{E}[\bar{X}^{s,a}]| \leq  \varepsilon\right)$$
Using the independance of our samples, we have :

\begin{equation}
\begin{aligned}
\mathbb{P}\left(\max_{s,a} |\bar{X}^{s,a} - \mathbb{E}[\bar{X}^{s,a}]| \leq  (1-\gamma)\varepsilon\right) &= \mathbb{P}\left(\bigcap_{s,a} \left[|\bar{X}^{s,a} - \mathbb{E}[\bar{X}^{s,a}]| \leq  (1-\gamma)\varepsilon\right]\right)\\ &= \prod_{s,a} \mathbb{P}\left(|\bar{X}^{s,a} - \mathbb{E}[\bar{X}^{s,a}]| \leq  (1-\gamma)\varepsilon\right) \\
&\geq  \left( 1 - 2e^{-2(1-\gamma)^4N\varepsilon^2}\right)^{|S||A|}
\end{aligned}
\end{equation}
The last inequality comming from the Hoeffding's inequality of above. We can then take :
$$\varepsilon = \sqrt{- \frac{\log \left(\frac{1 - (1 - \delta)^{\frac{1}{|S||A|}}}{2}\right)}{2(1-\gamma)^4N}}$$
we finally have :
$$\mathbb{P}\left(\|\hat{V} - V^* \|_{\infty} \leq  \varepsilon\right) \geq 1 - \delta$$


## Question 2.3

Suppose once again that we are given a dataset of $N$ samples in the form of tuples $(s_i,a_i,s_i',r_i)$. We know that each tuple contains a valid transition from the true MDP, i.e., $s_i' \sim P(\cdot | s_i, a_i)$ and $r_i \sim R(s_i,a_i)$, while the state-action pairs $(s_i,a_i)$ from which the transition started can be arbitrary.

Suppose we want to apply Q-learning to this MDP. Can you think of a way to leverage this offline data to improve the sample-efficiency of the algorithm? What if we were using SARSA instead?

### **Answer**

- For the Q-learning, we can use the Tabular Dyna-Q algorithm to have a better sample-efficiency. In this algorithm, the main idea is to plan with Model-based learning andto learning with Model-free learning. In practice, we update our model at each step, so that we are closer to the real one and thus improve the sample-efficiency. 
- For SARSA, we can use an importance sampling methode to improve the sample-efficiency. This means that we use a seighted distribution that helps us favour "important" samples over less important ones. 


# Part 3 - RL with Function Approximation

## Question 3.1

Given a datset $(s_i, a_i, r_i, s_i')$ of (states, actions, rewards, next states), the Fitted Q-Iteration (FQI) algorithm proceeds as follows:


* We start from a $Q$ function $Q_0 \in \mathcal{F}$, where $\mathcal{F}$ is a function space;
* At every iteration $k$, we compute $Q_{k+1}$ as:

$$
Q_{k+1}\in\arg\min_{f\in\mathcal{F}} \frac{1}{2}\sum_{i=1}^N
\left(
  f(s_i, a_i) - y_i^k
\right)^2 + \lambda \Omega(f)
$$
where $y_i^k = r_i + \gamma \max_{a'}Q_k(s_i', a')$, $\Omega(f)$ is a regularization term and $\lambda > 0$ is the regularization coefficient.


Consider FQI with *linear* function approximation. That is, for a given feature map $\phi : S \rightarrow \mathbb{R}^d$, we consider a parametric family of $Q$ functions $Q_\theta(s,a) = \phi(s)^T\theta_a$ for $\theta_a\in\mathbb{R}^d$. Suppose we are applying FQI on a given dataset of $N$ tuples of the form $(s_i, a_i, r_i, s_i')$ and we are at the $k$-th iteration. Let $\theta_k \in\mathbb{R}^{d \times A}$ be our current parameter. Derive the *closed-form* update to find $\theta_{k+1}$, using $\frac{1}{2}\sum_a ||\theta_a||_2^2$ as regularization.

### **Answer**

According that we consider the FQI with linear function approximation we can rewrite the defintion of $Q_{k+1}$ as :

$$Q_{k+1} \in \arg\min_{\theta \in \mathbb{R}^{d\times A}} \frac{1}{2} \sum_{i=1}^N \left( \phi(s_i)^T \theta_{a_i} - y_i^k \right)^2 + \frac{1}{2}\lambda \sum_{i=1}^N \| \theta_a\|_2^2$$
Let us define 
$$\forall \theta \in \mathbb{R}^{d\times A}, ~\Psi(\theta) =  \frac{1}{2} \sum_{i=1}^N \left( \phi(s_i)^T \theta_{a_i} - y_i^k \right)^2 + \frac{1}{2}\lambda \sum_{i=1}^N \| \theta_a\|_2^2$$
To find the minimum of $\Psi$, let $a$ be a action. We will compute the gradient of $\Psi$ according to $a$ : $\nabla_{\theta_a}\Psi$ : 
$$\nabla_{\theta_a}\Psi(\theta)  = \sum\limits_{\substack{i=0 \\ s.t.~a_i = a}}^N \left( (\phi(s_i)^T \theta_{a_i} - y_i^k )\phi(s_i) + \lambda \theta_{a_i} \right) $$
So, 
$$\nabla_{\theta_a}\Psi(\theta) = 0 \iff \theta_a = \left(\sum\limits_{\substack{i=0 \\ s.t. a_i = a}}^N \phi(s_i)\phi(s_i)^T + \lambda I_d \right)^{-1} \sum\limits_{\substack{i=0 \\ s.t.~a_i = a}}^N y_i^k \phi(s_i)$$

So, we have $\theta_{k+1}$ such that, for all $a$,
$$\theta_{k+1,a} = \left(\sum\limits_{\substack{i=0 \\ s.t. a_i = a}}^N \phi(s_i)\phi(s_i)^T + \lambda I_d \right)^{-1} \sum\limits_{\substack{i=0 \\ s.t.~a_i = a}}^N y_i^k \phi(s_i)$$
And, $$Q_{k+1}(s,a) = \phi(s)^T\theta_{k+1,a}$$

## Question 3.2

The code below creates a larger gridworld (with more states than the one used in the previous questions), and defines a feature map. Implement linear FQI to this environment (in the function `linear_fqi()` below), and compare the approximated $Q$ function to the optimal $Q$ function computed with value iteration.

Can you improve the feature map in order to reduce the approximation error?

### **Answer**

[explanation about how you tried to reduce the approximation error + FQI implementation below]

In [ ]:
def get_large_gridworld():
  """Creates an instance of a grid-world MDP with more states."""
  walls = [(ii, 10) for ii in range(15) if (ii != 7 and ii != 8)]
  env = GridWorld(
      nrows=15,
      ncols=15,
      reward_at = {(14, 14):1.0},
      walls=tuple(walls),
      success_probability=0.9,
      terminal_states=((14, 14),)
  )
  return env


class GridWorldFeatureMap:
  """Create features for state-action pairs
  
  Args:
    dim: int
      Feature dimension
    sigma: float
      RBF kernel bandwidth
  """
  def __init__(self, env, dim=15, sigma=0.25):
    self.index2coord = env.index2coord
    self.n_states = env.Ns
    self.n_actions = env.Na
    self.dim = dim
    self.sigma = sigma

    n_rows = env.nrows
    n_cols = env.ncols

    # build similarity matrix
    sim_matrix = np.zeros((self.n_states, self.n_states))
    for ii in range(self.n_states):
        row_ii, col_ii = self.index2coord[ii]
        x_ii = row_ii / n_rows
        y_ii = col_ii / n_cols
        for jj in range(self.n_states):
            row_jj, col_jj = self.index2coord[jj]
            x_jj = row_jj / n_rows
            y_jj = col_jj / n_cols
            dist = np.sqrt((x_jj - x_ii) ** 2.0 + (y_jj - y_ii) ** 2.0)
            sim_matrix[ii, jj] = np.exp(-(dist / sigma) ** 2.0)

    # factorize similarity matrix to obtain features
    uu, ss, vh = np.linalg.svd(sim_matrix, hermitian=True)
    self.feats = vh[:dim, :]

  def map(self, observation):
    feat = self.feats[:, observation].copy()
    return feat

In [ ]:
env = get_large_gridworld()
feat_map = GridWorldFeatureMap(env)

# Visualize large gridworld
render_policy(env)

# The features have dimension (feature_dim).
feature_example = feat_map.map(1) # feature representation of s=1
print(feature_example)

# Initial vector theta representing the Q function
theta = np.zeros((feat_map.dim, env.action_space.n))
print(theta.shape)
print(feature_example @ theta) # approximation of Q(s=1, a)

In [ ]:
def linear_fqi(env, feat_map, num_iterations, lambd=0.1, gamma=0.95):
  """
  # Linear FQI implementation
  # TO BE COMPLETED
  """

  N = 5000
  Ns, Na = env.R.shape

  # get a dataset
  dataset = get_uniform_dataset(env, n_samples = N)
  # dataset = get_random_policy_dataset(env, n_samples=...)
  (states, actions, rewards, next_states) = dataset

  theta = np.zeros((feat_map.dim, env.Na))
  Rmax = np.max(rewards)
  for it in range(num_iterations):
    #Compute y^k_i :
    y_k = np.zeros(N)
    for i in range(N):
      Q_k = [np.sum(feat_map.map(next_states[i])*theta[:,a]) for a in range(Na)]
      f = np.max(Q_k)
      #clipping 
      if f > 0 :
        f = min(f , Rmax / (1-gamma) )
      else : 
        f_s_a = max(f, - Rmax/(1-gamma))

      y_k[i] = rewards[i] + gamma * f
    
    #Create theta_{k+1}
    theta_new = np.zeros((feat_map.dim, env.Na))

    #For each action a
    for a in range(Na):
      #First we want to get the indices i such that a_i = a:
      #indices = np.where(np.array(actions) == a)[0]

      indices = []
      for i in range(N):
        if actions[i] == a:
          indices.append(i)

      if len(indices) != 0:
        to_be_inv = np.sum([np.matmul(np.array([feat_map.map(states[i])]).T, np.array([feat_map.map(states[i])])) for i in indices],axis = 0) + lambd*np.eye(feat_map.dim)

        #to_be_inv = np.sum([np.matmul(np.transpose(np.asmatrix(feat_map.map(states[i]))),np.asmatrix(feat_map.map(states[i]))) for i in indices],axis=0) + lambd*np.eye(feat_map.dim)
        #print(to_be_inv.shape)
        sum = np.sum([y_k[i]*feat_map.map(states[i]) for i in indices],axis=0)
        #print(sum.shape)
        theta_new[:,a] = np.matmul(np.linalg.inv(to_be_inv),sum)
    
    theta = np.copy(theta_new)
  
  return theta


# ----------------------------
# Environment and feature map
# ----------------------------
env = get_large_gridworld()
# you can change the parameters of the feature map, and even try other maps!
feat_map = GridWorldFeatureMap(env, dim=50, sigma=0.25)

# -------
# Run FQI
# -------
theta = linear_fqi(env, feat_map, num_iterations=100)

# Compute and run greedy policy
Q_fqi = np.zeros((env.Ns, env.Na))
for ss in range(env.Ns):
  state_feat = feat_map.map(ss)
  Q_fqi[ss, :] = state_feat @ theta

V_fqi = Q_fqi.max(axis=1)
policy_fqi = Q_fqi.argmax(axis=1)
render_policy(env, policy_fqi, horizon=100)

# Visualize the approximate value function in the gridworld.
img = env.get_layout_img(V_fqi)    
plt.imshow(img)
plt.show()



In [ ]:
#Comparaison betwenn Q_fqi and Q from the value iteration
VI_Q, VI_greedypol, all_qfunctions, number_it_VI = value_iteration(env.P, env.R)

mae_Q = mean_absolute_error(VI_Q,Q_fqi)  
mse_Q = mean_squared_error(VI_Q,Q_fqi)  
print("Mean square error for Q : ", mse_Q)
print("Mean absolute error for Q : ", mae_Q)


We see that the mean squared error and the mean absolute error between the  approximated $Q$ function to the optimal $Q$ function computed with value iteration are quite low. But when we compare the value maps from the value iteration and the one from function approximation, we can see that they quite lookalike but the function appromixation one has higher values ferther away from the objectif. So qualitatively, the red diamond will find its way to the objectif, but the value function will be slightly higher. 

In [ ]:
V_VI = VI_Q.max(axis=1)
plt.subplot(1,2,1)
img_VI = env.get_layout_img(V_VI)    
plt.imshow(img_VI)
plt.title("Value iteration")
plt.subplot(1,2,2)
plt.imshow(img)
plt.title("Function approximation")
plt.suptitle("Value maps")
plt.show()

In [ ]:
#Influence of sigma and dim : 
sigma = np.linspace(0.1,1,10)
dim = np.linspace(10,100,10,dtype=int)

mse = np.zeros((10,10))
mae = np.zeros((10,10))

VI_Q, VI_greedypol, all_qfunctions, number_it_VI = value_iteration(env.P, env.R)


for s in range(10):
  for d in range(10):
    feat_map = GridWorldFeatureMap(env, dim=dim[d], sigma=sigma[s])

    theta = linear_fqi(env, feat_map, num_iterations=100)

    # Compute and run greedy policy
    Q_fqi = np.zeros((env.Ns, env.Na))
    for ss in range(env.Ns):
      state_feat = feat_map.map(ss)
      Q_fqi[ss, :] = state_feat @ theta


    mae[d,s] = (mean_absolute_error(VI_Q,Q_fqi))
    mse[d,s] = (mean_squared_error(VI_Q,Q_fqi))    


In [ ]:
plt.subplot(1,2,1)
plt.pcolormesh(mae)
plt.colorbar()
plt.xlabel("sigma")
plt.ylabel("Dim")
plt.title("Mean absolute error")


plt.subplot(1,2,2)
plt.pcolormesh(mse)
plt.colorbar()
plt.xlabel("sigma")
plt.ylabel("Dim")
plt.title("Mean squared error")

plt.suptitle("Error between Q for value iteration and Q from function approximation")
plt.show()



I have computed the MSE and MAE between Q for value iteration and Q from function approximation for different values of the dimension $d$ of $\phi$ and $\sigma$. This way, I hope finding the best combination of those two values in order to minimize this error. The minimum is obtained for $d = 80$ and $\sigma = 0.1$. 